Importing pandas ; Importing the final progress dataframe ; Importing seaborn ; Importing numpy ; Importing the average closing price ; Importing the price csv ; Setting price csv index ; Preparing the final dataframe for prediction

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

df_close=pd.read_csv('../../data/output/average-closing-price.csv',sep=',')
df_close=df_close.set_index(['Year'])

df_price=pd.read_csv('../../data/input/price.csv',sep=',')
df_price['Year']=pd.to_datetime(df_price['Date']).dt.year
df_price['Month']=pd.to_datetime(df_price['Date']).dt.month
df_price=df_price.set_index(['Year','Month']).drop(columns=['Date','Adj Close','Volume'])

df_final=pd.read_csv('../../data/output/monthly-progress-by-book-value/mean.csv',sep=';')[['Year','Month','AR 5 Mean']]
df_final=df_final.set_index(['Year','Month'])

df_final=pd.merge(df_price,df_final,how='left',left_index=True,right_index=True)
df_final=pd.merge(df_final,df_close,how='left',left_index=True,right_index=True)
df_final=df_final.dropna()
df_final['Estimated Price']=df_final['AR 5 Mean']*df_final['Closing Price']
df_final=df_final.drop(columns=['AR 5 Mean','Closing Price'])

print(df_final.head())

              Open    High     Low   Close  Estimated Price
Year Month                                                 
2017 1      3100.0  3100.0  3100.0  3100.0      3472.823643
     1      3100.0  3155.0  3045.0  3155.0      3472.823643
     1      3170.0  3175.0  3130.0  3140.0      3472.823643
     1      3140.0  3155.0  3125.0  3135.0      3472.823643
     1      3090.0  3140.0  3090.0  3120.0      3472.823643


Prompting investor buy and sell deviation from AR mean

In [2]:
# the deviation from error mean
deviation_mean=0.029067
deviation_buy=0.01
deviation_sell=0.01

# security tax
security_tax=0.004

Defining profitability apply function

In [3]:
hold_stock=False
day_temp=0
days=np.array([])

def determine_day_profit(row):
    global hold_stock
    global days
    global day_temp
        
    buy_price=row['Estimated Price']*(1-deviation_buy)
    sell_price=row['Estimated Price']*(1+deviation_sell)
    if hold_stock:
        day_temp+=1
        if sell_price<=row['High']:
            days=np.append(days,day_temp)
            day_temp=0
            hold_stock=False
            return True
    else:
        if buy_price>=row['Low']:
            hold_stock=True
            
    return False

Applying profitability function to final dataframe

In [4]:
df_final['Profit']=df_final.apply(lambda row: determine_day_profit(row), axis=1)

Cleaning the days

In [5]:
Q1=np.percentile(days,25)
Q3=np.percentile(days,75)
IQR=Q3-Q1
lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR
days=days[(days>=lower_bound)&(days<=upper_bound)]

print("With a profit of",(deviation_buy+deviation_sell-security_tax)*100,"% per transaction:")
print("There is",df_final['Profit'].value_counts()[True],"profitable day of",len(df_final),"days.")
print("The mean of profitable days is",np.mean(days))

With a profit of 1.6 % per transaction:
There is 45 profitable day of 1710 days.
The mean of profitable days is 12.634146341463415
